# Process for cleaning url lists from SQL database

## Step 0. Extract lists of urls for training sets from SQL database

1. The data is on pocitac database, so you can connect via MySQLWorkbench
1. Select the PVC database
1. There are 4 tables of interest, two with 'manual' in the name and two with 'approved'.  The manual are highest confidence, but we will use the approved ones because they include all the manual ones, and in the end, they are all suppressed.  This will give us a larger data set.  It is about 2.3 to 3k for the manual sets, but it is near 22 to 30k for the approved sets.

## These are the raw queries for the staffing and tech_consult lists
```SQL
# These queries are repeated for staffing and tech_consult
# select the pcv database!

# good ones, around 3k
select *
from manual_suppress_on_url
where reason='no_suppress'

# 170810 udpated good ones, around 62k!
select output_url 
from output 
group by output_url
having min(primary_business_prod)=1
limit 100000
;

# staffing, around 22k
select *
from approved_suppress_on_url
where reason='staffing'
;

# tech_consult, around 30k
select *
from approved_suppress_on_url
where reason='tech_consult'
;
```

## Export query results as csv, which will be in this format
| id | url       | reason       | origin       | product_hits | added_at   |
|----|-----------|--------------|--------------|--------------|------------|
|  1 | acme.com  | tech_consult |  manual_jodi | 0            | 2016-09-09 |



## 1 Extract single column url list from SQL output

In [3]:
"""
Extract data from csv files which are provided from the SQL database.

File Input format is described in last step
File Output format is a txt file with a single column one url per line


inputs:
    source_name, Filename/path of file containing raw url lists from database.
    output_name, Filename to use for file which will contain only the urls
return:
    None
"""
import re
import pandas as pd

filename = 'approved_suppress_staffing_170913.csv'
input_path = 'data/00_source_urls'
output_path = 'data/01_clean_url_lists'

source_name = "{}/{}".format(input_path, filename)
output_name = "{}/{}.txt".format(output_path, re.sub('\.\w*$', '',filename))



print "Reading input file and extracting the url vertical to save to file"
df = pd.read_csv(source_name)
df[['url']].to_csv(output_name, index=False) # index=False keeps it from writing out the row number...

print "Printing out the first rows of the output file for verification"
df_out = pd.read_csv(output_name)
df_out.head()

print "Saving result to ", output_name
print "ALL DONE"
df_out.head()


Reading input file and extracting the url vertical to save to file
Printing out the first rows of the output file for verification
Saving result to  data/01_clean_url_lists/approved_suppress_staffing_170913.txt
ALL DONE


,url
0,aghires.com
1,archirings.com
2,barona.fi
3,betterhire.com
4,bluecubeit.com


# 2 Download and save raw data html content for the url lists

In [1]:
# List available url lists for next step
!ls data/01_clean_url_lists


62k_new_good_ones.txt                 test.txt
approved_suppress_staffing_170913.txt test_staff.txt


In [3]:
# Remove all known foreign language country url endings
def is_foreign_url_extension(url):
    foreign_extensions = [".mx", ".es", ".ar", ".cl", # spanish
                          ".se", # swedish
                          ".fr", ".tn", # french
                          ".dk", # danish
                          ".nl", ".be", # dutch
                          ".cz", # czech
                          ".it", # italian
                          ".br", ".pt", # portugese
                          ".de", ".ch", ".at", # german
                          ".no", # norwegian
                          ".pl", # polish
                          ".tr", # turkish
                          ]
    for ext in foreign_extensions:
        if url.endswith(ext):
            return True
    return False

source_file = 'data/170915_manual_tech_consult.csv'
target_file = 'data/170915_manual_tech_consult_noforext.csv'
with open(source_file, 'r') as source, open(target_file, 'w') as target:
    for line in source:
        url = line.strip()
        if not is_foreign_url_extension(url):
            target.write(url)
            target.write('\n')
            
print "DONE: Results written to ", target_file
    


DONE: Results written to  data/170915_manual_tech_consult_noforext.csv


In [6]:
%%bash
# if you see this message: dyld: Library not loaded: /usr/local/opt/readline/lib/libreadline.6.dylib
# you need to upgrade bash via `brew upgrade bash`, then restart the kernel

## Run the output list from previous cell through the url-classifier in text saving mode.
#. ../set_flag/return_full_text_on               # This sets an env variable which tells urlChecker to save content to disk
export URL_FULL_CONTENT=True

cat data/170915_manual_tech_consult_noforext.csv | \
 python ../bin/urlChecker.py > \
 data/170915_manual_tech_consult_noforext_content.tsv
echo "DONE!"
### When the script finishes, you should have an html file in the `URL_SAVE_DEST` location named after the url, i.e. acme.com will have an acme.com file with the html content in it.

DONE!


----------
volansys.com
ciinc.com
pyramidci.com
bluecanopy.com
asd-inc.com
chreynolds.com
blytheco.com
azad.com
banglalink.com.bd
acis.co.in
analytica.net
cimarroninc.com
asd-web.com
adccinfocad.com
apextgi.in
cdac.in
apertuspartners.com
bylight.com
auxilioinc.com
andesaservices.com
alchemists-ark.com
cadgulf.com
centerbeam.com
cistel.com
4cplus.com
capstonetek.com
bridgeenergygroup.com
arena-multimedia.com
cat-amania.com
brennanit.com.au
asyst.co.id
c7.com
buzzclan.com
circlesolutions.com
calpinetech.com
chipstechnologygroup.com
ait.com
c-cor.com.au
betis.com
b3solutions.com
chrp-india.com
atypon.com
cogenteservices.com
capitolofficesolutions.com
axxumtech.com
chinetworks.com
bcfsolutions.com
askusa.com
it.arvato.com
access-company.com
deltek.com
crmit.com
aderant.com
ibackup.com
panaya.com
trinet.com
ntis.gov
wbmd.com
iskraemeco.com
melissadata.com
cts.wa.gov
thinkvine.com
delegosoftware.com
loc8.com
medevolve.com
backblaze.com
unanet.com
vision6.com.au
mycompas.com
checkmarx.com
inf

# 3 Extract vislble text, clean text and store in csv file in final form for classifers

In [9]:
"""
This script extracts readable content from a set of raw html files.  It takes as input a file with tuples.
Each tuple is a filename which containing just one column of urls,
and the second element is simply a classification to tag all urls with.  
It's output will be a file similarly named, and will contain three columns...
url, visible_text, class

inputs:
    sets, Array of tupples. Each tuple is a filepath/class.  File contails urls to extract content from.
    html_content, Filepath to directory containing raw html files for extraction
output:
    file will be generated with 'data_for_classification' appended to source name.
return:
    None
"""
import os
print os.getcwd()

#INPUTS:
sets = [
     ("data/01_clean_url_lists/test.txt", 0),
     ("data/01_clean_url_lists/test_staff.txt", 1),
]
html_content = '../html_content'
target_path = 'data/02_training_sheets'
silent = False
# END INPUTS

import classification.data_extraction_functions as de
for fpath, binary in sets:
    de.disk_content_using_file_list(fpath, html_content, target_path, binary, silent=silent)
print "ALL DONE"

/Users/efrainolivares/repos/url_classifier_eoStaffingCleanup/notebooks
Opening data/01_clean_url_lists/test.txt for urls
Writing data to data/02_training_sheets/test_0.tsv
0
Saved data to data/02_training_sheets/test_0.tsv
Opening data/01_clean_url_lists/test_staff.txt for urls
Writing data to data/02_training_sheets/test_staff_1.tsv
0
url low content welcome to www.4.cn
Saved data to data/02_training_sheets/test_staff_1.tsv
ALL DONE


In [10]:
!python -m pytest classification/data_extraction_functions.py -v

============================= test session starts ==============================
platform darwin -- Python 2.7.10, pytest-3.1.3, py-1.4.34, pluggy-0.4.0 -- /Users/efrainolivares/envnltk/bin/python
cachedir: .cache
rootdir: /Users/efrainolivares/repos/url_classifier_eoStaffingCleanup/notebooks, inifile:
collected 6 items 

classification/data_extraction_functions.py::test_lowercase PASSED
classification/data_extraction_functions.py::test_paragraph PASSED
classification/data_extraction_functions.py::test_script PASSED
classification/data_extraction_functions.py::test_style PASSED
classification/data_extraction_functions.py::test_style2 PASSED
classification/data_extraction_functions.py::test_clean_out_junk_words PASSED

=========================== 6 passed in 0.07 seconds ===========================
